# dice_contract

In [ ]:
open rust
open rust.rust_operators

In [ ]:
//// test

open testing

## dice_contract

### state

In [ ]:
type state =
    {
        version : u32
    }

In [ ]:
//// test
///! rust -c

()

20:31:44   debug #1 spiral_wasm.print_usd / { retry = 1; total_gas_burnt_usd = +0.000808; total_gas_burnt = 1209371440851 }
20:31:44   debug #2 spiral_wasm.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000206; tokens_burnt_usd = +0.000206; gas_burnt = 308081859340; tokens_burnt = 30808185934000000000 }
20:31:44   debug #3 spiral_wasm.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000602; tokens_burnt_usd = +0.000602; gas_burnt = 901289581511; tokens_burnt = 90128958151100000000 }
20:31:44 warning #4 spiral_wasm.run / Error error / { retry = 1; error = "{ receipt_failures = []; receipt_outcomes_len = 1; retry = 1 }" }
20:31:53   debug #5 spiral_wasm.print_usd / { retry = 2; total_gas_burnt_usd = +0.000957; total_gas_burnt = 1432554003351 }
20:31:53   debug #6 spiral_wasm.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000206; tokens_burnt_usd = +0.000206; gas_burnt = 308081859340; tokens_burnt = 30808185934000000000 }
20:31:53   debug #7 spir

### new

In [ ]:
inl new () : state =
    {
        version = 1
    }

In [ ]:
//// test
///! rust -c

inl state = new ()
trace Verbose (fun () => "dice_contract") fun () => { state }

00:00:00 verbose #1 dice_contract / { state = { version = 1 } }
20:32:15   debug #1 spiral_wasm.print_usd / { retry = 1; total_gas_burnt_usd = +0.001087; total_gas_burnt = 1627283487229 }
20:32:15   debug #2 spiral_wasm.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000206; tokens_burnt_usd = +0.000206; gas_burnt = 308081859340; tokens_burnt = 30808185934000000000 }
20:32:15   debug #3 spiral_wasm.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000732; tokens_burnt_usd = +0.000732; gas_burnt = 1096019065389; tokens_burnt = 109601906538900000000 }
20:32:15   debug #4 spiral_wasm.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000149; tokens_burnt_usd = +0.000000; gas_burnt = 223182562500; tokens_burnt = 0 }
